<a href="https://colab.research.google.com/github/salathegroup/multi-lang-vaccine-sentiment/blob/master/Create_Vaccine_Datasets_from_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Create Vaccine Datasets from Sheets
Creates the vaccine datasets both for the annotated and the unannotated stuff stored in the Google Sheets.

In the end it copies the training datasets to the Google Cloud.

In [156]:
#@markdown ##Read Sheets

import pandas as pd
import numpy as np
import gspread
import sys, os
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Read the annotated data
sh = gc.open('EPFL_vaccine_sentiment_3fold_agreed')
worksheet = sh.worksheet("unique")
rows = worksheet.get_all_values()

#Get it into pandas
import pandas as pd
annotated = pd.DataFrame.from_records(rows)
annotated.columns = annotated.iloc[0]
annotated = annotated.reindex(annotated.index.drop(0))
annotated['a'] = "a"


print("Successfully read the annotated data")
#Read the unannotated data
sh = gc.open('EPFL unannotated tweets')
worksheet = sh.worksheet("raw")
rows = worksheet.get_all_values()

#Get it into pandas
unannotated = pd.DataFrame.from_records(rows)
unannotated.columns = unannotated.iloc[0]
unannotated = unannotated.reindex(unannotated.index.drop(0))
unannotated['a'] = "a"
unannotated = unannotated[0:10000]

print("Successfully read the unannotated data")


Successfully read the annotated data
Successfully read the unannotated data


In [157]:
# #@markdown ##Create and Copy Data Files
import warnings
warnings.filterwarnings('ignore')

#annotated data
#Split into 60% train - 20% validation/development - 20% test
train   = annotated.loc[annotated["group"] == "train"]
train_small = train.sample(n=1000, random_state=42)

dev   = annotated.loc[annotated["group"] == "dev"]
test   = annotated.loc[annotated["group"] == "test"]

languages = ["cb-annot-en","cb-annot-en-fr","cb-annot-en-de","cb-annot-en-es","cb-annot-en-pt"]

if not os.path.exists('annotated'):
    os.makedirs('annotated')

for lang in languages:
  #Save annotated (large)
  if not os.path.exists(lang):
    os.makedirs(lang)
  
  with open('/content/'+lang+"/train.tsv", 'w') as f:
    f.write(train.to_csv(columns=["id", "label", "a", lang], header=False, index=False, sep='\t'))

  with open('/content/'+lang+"/dev.tsv", 'w') as f:
    f.write(dev.to_csv(columns=["id", "label", "a", lang], header=False, index=False, sep='\t'))

  with open('/content/'+lang+"/test.tsv", 'w') as f:
    f.write(test.to_csv(columns=["id", "label"], header=False, index=False, sep='\t'))

  with open('/content/'+lang+"/annotated_test.tsv", 'w') as f:
    f.write(test.to_csv(columns=["id", "label", "a", lang], header=False, index=False, sep='\t'))

  #Save annotated small
  if not os.path.exists(lang+'-sm'):
      os.makedirs(lang+'-sm')
    
  with open('/content/'+lang+'-sm/train.tsv', 'w') as f:
    f.write(train_small.to_csv(columns=["id", "label", "a", lang], header=False, index=False, sep='\t'))

  with open('/content/'+lang+'-sm/dev.tsv', 'w') as f:
    f.write(dev.to_csv(columns=["id", "label", "a", lang], header=False, index=False, sep='\t'))

  with open('/content/'+lang+'-sm/test.tsv', 'w') as f:
    f.write(test.to_csv(columns=["id", "label"], header=False, index=False, sep='\t'))

  with open('/content/'+lang+'-sm/annotated_test.tsv', 'w') as f:
    f.write(test.to_csv(columns=["id", "label", "a", lang], header=False, index=False, sep='\t'))


#unannotated data
languages = ["cb-en","cb-en-fr","cb-en-de","cb-en-es"]

if not os.path.exists('cb-unannotated'):
    os.makedirs('cb-unannotated')

for lang in languages:  
  with open('/content/cb-unannotated/'+lang+".tsv", 'w') as f:
    f.write(unannotated.to_csv(columns=["id", lang], header=False, index=False, sep='\t'))

print("Created the data files locally")


#Copy everything to the bucket
!gsutil -m cp -r cb-*/ gs://perepublic/EPFL_multilang/data/

print("Copied the data to the bucket")


Created the data files locally
Copying file://cb-annot-en/dev.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de/test.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en/test.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en/annotated_test.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de/annotated_test.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en/train.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de/train.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de/dev.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de-sm/dev.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de-sm/test.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de-sm/annotated_test.tsv [Content-Type=text/tab-separated-values]...
Copying file://cb-annot-en-de-